## Loading Data


In [ ]:
!pip install codecarbon catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.9/494.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 23.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import RandomUnderSampler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import json
import os
import logging
import sys
from codecarbon import EmissionsTracker

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df= pd.read_csv('/content/drive/MyDrive/Scriptie uitvoeringsfase (workfolder) Uni/Processed data/pa_cg.csv')

In [ ]:
df.shape

(538053, 41)

In [ ]:
df.drop(columns=['Unnamed: 0','VKL_NUMMER'],axis=1,inplace=True)

In [ ]:
nan_rows = df[df['WGD_CODE_1'].isna()]

# Report the number of rows with SEVERE=1 and NaN in 'WGD_CODE_1'
severe_nan_rows = nan_rows[nan_rows['SEVERE'] == 1]
deleted_count = len(severe_nan_rows)
print(f"Number of rows with SEVERE=1 and NaN in 'WGD_CODE_1': {deleted_count}")
print(f"Total rows before deletion: {df.shape[0]}")

# Delete rows with NaN values in 'WGD_CODE_1'
df = df.dropna(subset=['WGD_CODE_1'])

# Display the cleaned DataFrame
print(f"Total rows after deletion: {df.shape[0]}")

Number of rows with SEVERE=1 and NaN in 'WGD_CODE_1': 1579
Total rows before deletion: 538053
Total rows after deletion: 523186


## Models

In [ ]:
# Create a custom logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# Create handlers
stream_handler = logging.StreamHandler(sys.stdout)
stream_handler.setLevel(logging.DEBUG)

# Create formatters and add it to handlers
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
stream_handler.setFormatter(formatter)

# Add handlers to the logger
if not logger.handlers:
    logger.addHandler(stream_handler)

# Test logging to ensure it's working correctly
logger.debug("This is a debug message.")
logger.info("This is an info message.")
logger.warning("This is a warning message.")
logger.error("This is an error message.")
logger.critical("This is a critical message.")
sys.stdout.flush()


DEBUG:root:This is a debug message.
INFO:root:This is an info message.
ERROR:root:This is an error message.
CRITICAL:root:This is a critical message.


In [ ]:
# Define random state for reproducibility
random_state = 42

# Define column categories
nominal_columns = ['MNE_CODE', 'AOL_ID', 'BEBKOM', 'WVL_ID', 'WVG_ID', 'WDK_ID', 'WSE_ID', 'WGD_CODE_1', 'sorted_hexbin_id', 'YEAR', 'TYPE', 'PVE_CODE']
ordinal_columns = ['Daylight_ID', 'Min Age Group', 'Max Age Group', 'Mode Age Group']
numerical_columns = ['MeanWindSpeed_mps', 'MaxHourlyWindSpeed_mps', 'MaxWindGust_mps', 'MeanTemperature_C', 'MinTemperature_C', 'MaxTemperature_C', 'SunshineDuration_hrs', 'PrecipitationDuration_hrs', 'MeanSLPressure_hPa', 'MaxVisibility_km', 'MeanCloudCover_oct', 'Ratio_male', 'Ratio_female', 'MAXSNELHD']
log_columns = ['TotalDailyPrecip_mm']
binary_columns = ['IND_ALC', 'WEEKEND']
hour_column = ['HOUR']
day_column = ['WEEKDAY']
month_column = ['MONTH']

# Ensure all columns are categorized
all_columns = set(df.columns)
categorized_columns = set(nominal_columns + ordinal_columns + numerical_columns + log_columns + hour_column + day_column + month_column + binary_columns)
uncategorized_columns = all_columns - categorized_columns
logger.info(f"Uncategorized columns: {uncategorized_columns}")
sys.stdout.flush()


INFO:root:Uncategorized columns: {'Stratify_Key', 'SEVERE', 'Combined Group'}


In [ ]:
# Define cyclical encoding function
def cyclical_encode(df, max_val):
    return pd.DataFrame({
        df.columns[0] + '_sin': np.sin(2 * np.pi * df.iloc[:, 0] / max_val),
        df.columns[0] + '_cos': np.cos(2 * np.pi * df.iloc[:, 0] / max_val)
    })

hour_transformer = FunctionTransformer(cyclical_encode, kw_args={'max_val': 24}, validate=False)
weekday_transformer = FunctionTransformer(cyclical_encode, kw_args={'max_val': 7}, validate=False)
month_transformer = FunctionTransformer(cyclical_encode, kw_args={'max_val': 12}, validate=False)

# Fit encoders on the entire dataset
one_hot_encoder = OneHotEncoder(handle_unknown='ignore',sparse_output=False)
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

df_nominal = df[nominal_columns]
df_ordinal = df[ordinal_columns]

one_hot_encoder.fit(df_nominal)
ordinal_encoder.fit(df_ordinal)

OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('nom', one_hot_encoder, nominal_columns),
        ('ord', ordinal_encoder, ordinal_columns),
        ('num', StandardScaler(), numerical_columns),
        ('log', FunctionTransformer(np.log1p), log_columns),
        ('hour_cyc', hour_transformer, hour_column),
        ('weekday_cyc', weekday_transformer, day_column),
        ('month_cyc', month_transformer, month_column),
        ('bin', 'passthrough', binary_columns)
    ],
    remainder='drop'
)

In [ ]:


# Define models
models = [
    ('Decision Tree', DecisionTreeClassifier(random_state=random_state)),
    ('Random Forest', RandomForestClassifier(random_state=random_state)),
    ('Logistic Regressor', LogisticRegression(random_state=random_state, max_iter=10000)),
    ('Naive Bayes', GaussianNB()),
    ('ANN One Layer', MLPClassifier(hidden_layer_sizes=(10,), random_state=random_state, max_iter=1000)),
    ('ANN Two Layers', MLPClassifier(hidden_layer_sizes=(10, 10), random_state=random_state, max_iter=1000)),
    ('ADABoost', AdaBoostClassifier(random_state=random_state)),
    ('XGBoost', XGBClassifier(random_state=random_state)),
    ('L-GBM', LGBMClassifier(random_state=random_state)),
    ('CatBoost', CatBoostClassifier(random_state=random_state))
]



In [ ]:

# Splitting the data into train and test sets
train_data_full, test_data = train_test_split(df, test_size=0.20, stratify=df['Stratify_Key'], random_state=random_state)

# Save the test set
output_dir = '/content/drive/MyDrive/Scriptie uitvoeringsfase (workfolder) Uni/Results V37 (Dense K-Fold)'
os.makedirs(output_dir, exist_ok=True)
test_data.to_csv(f'{output_dir}/test_set.csv', index=False)

# Drop the 'Stratify_key' and 'Combined Group' from the test set
test_data = test_data.drop(columns=['Stratify_Key', 'Combined Group'])

# Drop the 'Stratify_key' and 'Combined Group' from the full training set as well
full_training_set = train_data_full.drop(columns=['Stratify_Key', 'Combined Group'])

# Identifying categorical features for SMOTENC using nominal_columns
categorical_features = [full_training_set.columns.get_loc(col) for col in nominal_columns]


In [ ]:
# Function to create resampled datasets
def get_resampled_sets(X, y, categorical_features):
    # Original set
    original_set = (X, y)

    # Undersampled set
    rus = RandomUnderSampler(sampling_strategy='auto', random_state=random_state)
    X_undersampled, y_undersampled = rus.fit_resample(X, y)

    # Oversampled set using SMOTENC
    smote_nc = SMOTENC(categorical_features=categorical_features, sampling_strategy='auto', random_state=random_state)
    X_oversampled, y_oversampled = smote_nc.fit_resample(X, y)

    return {
        'original': original_set,
        'undersampled': (X_undersampled, y_undersampled),
        'oversampled': (X_oversampled, y_oversampled)
    }

# Function to train and evaluate the model using k-fold cross-validation
def train_and_evaluate_model(preprocessor, model, model_name, X, y, test_data, output_dir, categorical_features, group_name, n_splits=3):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    all_results = []

    for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
        fold+=1
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        for resample_type, (X_resampled, y_resampled) in get_resampled_sets(X_train, y_train, categorical_features).items():
            # Initialize the emissions tracker for each resample type
            tracker = EmissionsTracker(output_dir=output_dir, log_level='error')
            tracker.start()

            pipeline = Pipeline(steps=[
                ('preprocessor', preprocessor),
                ('classifier', model)
            ])

            logger.info(f"Fitting the model using {resample_type} data for fold {fold}...")
            sys.stdout.flush()
            pipeline.fit(X_resampled, y_resampled)
            logger.info("Model fitting completed.")
            sys.stdout.flush()

            # Validation set evaluation
            y_pred_val = pipeline.predict(X_val)
            y_pred_val_proba = pipeline.predict_proba(X_val)[:, 1] if hasattr(pipeline, "predict_proba") else pipeline.decision_function(X_val)
            report_val = classification_report(y_val, y_pred_val, output_dict=True)
            auc_val = roc_auc_score(y_val, y_pred_val_proba)
            cm_val = confusion_matrix(y_val, y_pred_val)
            all_results.append({
                'model': model_name,
                'group': group_name,
                'resample_type': resample_type,
                'evaluation_type': 'validation',
                'fold': fold,
                'classification_report': report_val,
                'auc': auc_val,
                'confusion_matrix': cm_val.tolist()
            })

            val_predictions = pd.DataFrame({
                'Actual': y_val,
                'Predicted': y_pred_val,
                'Predicted_Proba': y_pred_val_proba
            })
            val_predictions.to_csv(f'{output_dir}/{model_name}_predictions_{group_name}_{resample_type}_validation_fold_{fold}.csv', index=False)

            # Test set evaluation
            logger.info(f"Evaluating {model_name} on the full test set using {resample_type} data for fold {fold}...")
            sys.stdout.flush()
            X_test_preprocessed = pipeline.named_steps['preprocessor'].transform(test_data.drop(columns=['SEVERE']))
            y_test = test_data['SEVERE']
            y_pred_test = pipeline.named_steps['classifier'].predict(X_test_preprocessed)
            y_pred_test_proba = pipeline.named_steps['classifier'].predict_proba(X_test_preprocessed)[:, 1] if hasattr(pipeline.named_steps['classifier'], "predict_proba") else pipeline.named_steps['classifier'].decision_function(X_test_preprocessed)
            report_test = classification_report(y_test, y_pred_test, output_dict=True)
            auc_test = roc_auc_score(y_test, y_pred_test_proba)
            cm_test = confusion_matrix(y_test, y_pred_test)
            all_results.append({
                'model': model_name,
                'group': group_name,
                'resample_type': resample_type,
                'evaluation_type': 'test',
                'fold': fold,
                'classification_report': report_test,
                'auc': auc_test,
                'confusion_matrix': cm_test.tolist()
            })

            test_predictions = pd.DataFrame({
                'Actual': y_test,
                'Predicted': y_pred_test,
                'Predicted_Proba': y_pred_test_proba
            })
            test_predictions.to_csv(f'{output_dir}/{model_name}_predictions_{group_name}_{resample_type}_test_fold_{fold}.csv', index=False)

            # Stop the emissions tracker and get the emissions data
            emissions = tracker.stop()
            emissions_data = {
                'emissions_kg_CO2eq': emissions,  # Total emissions in kgCO2eq
                'energy_consumed_kWh': tracker.final_emissions_data.energy_consumed,  # Total energy consumed in kWh
                'duration_seconds': tracker.final_emissions_data.duration,  # Total duration of the tracking in seconds
                'cpu_energy_kWh': tracker.final_emissions_data.cpu_energy,  # Energy consumed by the CPU in kWh
                'gpu_energy_kWh': tracker.final_emissions_data.gpu_energy,  # Energy consumed by the GPU in kWh
                'ram_energy_kWh': tracker.final_emissions_data.ram_energy,  # Energy consumed by the RAM in kWh
            }


        # Feature importance analysis
        if isinstance(model, LogisticRegression):
            feature_importance = abs(pipeline.named_steps['classifier'].coef_[0])

            # Get feature names after transformation
            one_hot_encoder = pipeline.named_steps['preprocessor'].named_transformers_['nom']
            one_hot_features = one_hot_encoder.get_feature_names_out(nominal_columns)
            ordinal_features = ordinal_columns
            numeric_features = numerical_columns + log_columns + hour_column + day_column + month_column + binary_columns
            feature_names = np.concatenate([one_hot_features, ordinal_features, numeric_features])

            feature_importance_dict = {name: importance for name, importance in zip(feature_names, feature_importance)}
            all_results[-1]['feature_importance'] = feature_importance_dict


            # Add emissions data to each result
            for result in all_results:
                if result['fold'] == fold and result['resample_type'] == resample_type:
                    result.update(emissions_data)

    # Save results
    results_file = f'{output_dir}/classification_reports.json'
    if os.path.exists(results_file):
        with open(results_file, 'r') as f:
            existing_results = json.load(f)
        all_results.extend(existing_results)

    with open(results_file, 'w') as f:
        json.dump(all_results, f, indent=4)

    results_df = pd.json_normalize(all_results)
    results_df.to_csv(f'{output_dir}/classification_reports.csv', index=False)
    logger.info(f"Results and predictions for {model_name} saved to {output_dir}")

# Function to create training sets for each group
def create_training_sets(train_data_full):
    train_eval_sets = {}
    # Full training set
    train_eval_sets['full_training_set'] = {
        'X': train_data_full.drop(columns=['SEVERE', 'Stratify_Key', 'Combined Group']),
        'y': train_data_full['SEVERE']
    }
    # Process each Combined Group
    groups = train_data_full['Combined Group'].unique()
    for group in groups:
        group_data = train_data_full[train_data_full['Combined Group'] == group].drop(columns=['Stratify_Key', 'Combined Group'])
        train_eval_sets[f'group_{group}'] = {
            'X': group_data.drop(columns=['SEVERE']),
            'y': group_data['SEVERE']
        }
    return train_eval_sets

train_eval_sets = create_training_sets(train_data_full)


In [ ]:
# Train and evaluate each model on each training set
for model_name, model in models:
    for group_name, dataset in train_eval_sets.items():
        logger.info(f"Starting evaluation for model: {model_name} on {group_name}")
        train_and_evaluate_model(preprocessor, model, model_name, dataset['X'], dataset['y'], test_data, output_dir, categorical_features, group_name)
        logger.info(f"Finished evaluation for model: {model_name} on {group_name}")

Streaming output truncated to the last 5000 lines.
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 169.254.169.254:80
DEBUG:urllib3.connectionpool:http://169.254.169.254:80 "GET /latest/dynamic/instance-identity/document HTTP/1.1" 400 1
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 169.254.169.254:80
DEBUG:urllib3.connectionpool:http://169.254.169.254:80 "GET /metadata/instance?api-version=2019-08-15 HTTP/1.1" 400 1
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 169.254.169.254:80
DEBUG:urllib3.connectionpool:http://169.254.169.254:80 "GET /computeMetadata/v1/instance/?recursive=true&alt=json HTTP/1.1" 200 36
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): get.geojs.io:443
DEBUG:urllib3.connectionpool:https://get.geojs.io:443 "GET /v1/ip/geo.json HTTP/1.1" 200 None
INFO:root:Fitting the model using undersampled data for fold 1...
INFO:root:Model fitting completed.
INFO:root:Evaluating ANN Two Layers on the full test set us

[LightGBM] [Info] Number of positive: 40458, number of negative: 238574
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019324 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2130
[LightGBM] [Info] Number of data points in the train set: 279032, number of used features: 152
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.144994 -> initscore=-1.774415
[LightGBM] [Info] Start training from score -1.774415


INFO:root:Model fitting completed.
INFO:root:Evaluating L-GBM on the full test set using original data for fold 1...
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 169.254.169.254:80
DEBUG:urllib3.connectionpool:http://169.254.169.254:80 "GET /latest/dynamic/instance-identity/document HTTP/1.1" 400 1
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 169.254.169.254:80
DEBUG:urllib3.connectionpool:http://169.254.169.254:80 "GET /metadata/instance?api-version=2019-08-15 HTTP/1.1" 400 1
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 169.254.169.254:80
DEBUG:urllib3.connectionpool:http://169.254.169.254:80 "GET /computeMetadata/v1/instance/?recursive=true&alt=json HTTP/1.1" 200 36
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): get.geojs.io:443
DEBUG:urllib3.connectionpool:https://get.geojs.io:443 "GET /v1/ip/geo.json HTTP/1.1" 200 None
INFO:root:Fitting the model using undersampled data for fold 1...


[LightGBM] [Info] Number of positive: 40458, number of negative: 40458
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006529 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2104
[LightGBM] [Info] Number of data points in the train set: 80916, number of used features: 143
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


INFO:root:Model fitting completed.
INFO:root:Evaluating L-GBM on the full test set using undersampled data for fold 1...
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 169.254.169.254:80
DEBUG:urllib3.connectionpool:http://169.254.169.254:80 "GET /latest/dynamic/instance-identity/document HTTP/1.1" 400 1
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 169.254.169.254:80
DEBUG:urllib3.connectionpool:http://169.254.169.254:80 "GET /metadata/instance?api-version=2019-08-15 HTTP/1.1" 400 1
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 169.254.169.254:80
DEBUG:urllib3.connectionpool:http://169.254.169.254:80 "GET /computeMetadata/v1/instance/?recursive=true&alt=json HTTP/1.1" 200 36
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): get.geojs.io:443
DEBUG:urllib3.connectionpool:https://get.geojs.io:443 "GET /v1/ip/geo.json HTTP/1.1" 200 None
INFO:root:Fitting the model using oversampled data for fold 1...


...

INFO:root:Evaluating CatBoost on the full test set using undersampled data for fold 2...
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 169.254.169.254:80
DEBUG:urllib3.connectionpool:http://169.254.169.254:80 "GET /latest/dynamic/instance-identity/document HTTP/1.1" 400 1
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 169.254.169.254:80
DEBUG:urllib3.connectionpool:http://169.254.169.254:80 "GET /metadata/instance?api-version=2019-08-15 HTTP/1.1" 400 1
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 169.254.169.254:80
DEBUG:urllib3.connectionpool:http://169.254.169.254:80 "GET /computeMetadata/v1/instance/?recursive=true&alt=json HTTP/1.1" 200 36
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): get.geojs.io:443
DEBUG:urllib3.connectionpool:https://get.geojs.io:443 "GET /v1/ip/geo.json HTTP/1.1" 200 None
INFO:root:Fitting the model using oversampled data for fold 2...


Learning rate set to 0.143456
0:	learn: 0.6386926	total: 42.9ms	remaining: 42.9s
1:	learn: 0.5210668	total: 88.6ms	remaining: 44.2s
2:	learn: 0.4945828	total: 138ms	remaining: 45.8s
3:	learn: 0.4736048	total: 186ms	remaining: 46.3s
4:	learn: 0.4584232	total: 227ms	remaining: 45.1s
5:	learn: 0.4143959	total: 272ms	remaining: 45s
6:	learn: 0.3973468	total: 312ms	remaining: 44.3s
7:	learn: 0.3788918	total: 358ms	remaining: 44.4s
8:	learn: 0.3709420	total: 400ms	remaining: 44.1s
9:	learn: 0.3632751	total: 441ms	remaining: 43.7s
10:	learn: 0.3542413	total: 486ms	remaining: 43.7s
11:	learn: 0.3505887	total: 526ms	remaining: 43.3s
12:	learn: 0.3468365	total: 570ms	remaining: 43.3s
13:	learn: 0.3422843	total: 619ms	remaining: 43.6s
14:	learn: 0.3393858	total: 664ms	remaining: 43.6s
15:	learn: 0.3304907	total: 705ms	remaining: 43.4s
16:	learn: 0.3261114	total: 746ms	remaining: 43.2s
17:	learn: 0.3244105	total: 787ms	remaining: 43s
18:	learn: 0.3229566	total: 830ms	remaining: 42.9s
19:	learn: 0.

INFO:root:Model fitting completed.


997:	learn: 0.1997599	total: 43.9s	remaining: 88ms
998:	learn: 0.1997392	total: 44s	remaining: 44ms
999:	learn: 0.1997225	total: 44s	remaining: 0us


INFO:root:Evaluating CatBoost on the full test set using oversampled data for fold 2...


In [ ]:
from google.colab import runtime
runtime.unassign()